In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
mnist_datasets, mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

In [4]:
mnist_train,mnist_test=mnist_datasets['train'],mnist_datasets['test']
num_validation_samples=0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)
num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)

def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255.
    return image,label

scaled_train_validation_data=mnist_train.map(scale)
test_data=mnist_test.map(scale)
BUFFER_SIZE=10000
shuffled_train_validation_data=scaled_train_validation_data.shuffle(BUFFER_SIZE)

validation_data=shuffled_train_validation_data.take(num_validation_samples)
train_data=shuffled_train_validation_data.skip(num_validation_samples)

BATCH_SIZE=100
train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)
validation_inputs,validation_targets=next(iter(validation_data))

In [5]:
input_size=784
output_size=10
hidden_layer_size=100

model=tf.keras.Sequential([
                           tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
    
])

In [6]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [7]:
NUM_EPOCHS= 5
model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),validation_steps=1,verbose=2)

Epoch 1/5
540/540 - 14s - loss: 0.3317 - accuracy: 0.9042 - val_loss: 0.1886 - val_accuracy: 0.9452
Epoch 2/5
540/540 - 11s - loss: 0.1417 - accuracy: 0.9578 - val_loss: 0.1177 - val_accuracy: 0.9623
Epoch 3/5
540/540 - 11s - loss: 0.0981 - accuracy: 0.9707 - val_loss: 0.0909 - val_accuracy: 0.9720
Epoch 4/5
540/540 - 10s - loss: 0.0760 - accuracy: 0.9773 - val_loss: 0.0765 - val_accuracy: 0.9785
Epoch 5/5
540/540 - 10s - loss: 0.0608 - accuracy: 0.9816 - val_loss: 0.0706 - val_accuracy: 0.9782


In [32]:
test_loss,test_accuracy=model.evaluate(test_data)
print('test_loss:{0:0.2f},test_accuracy:{1:0.2f}%'.format(test_loss,test_accuracy*100.))

      1/Unknown - 1s 1s/step - loss: 0.0861 - accuracy: 0.9725test_loss:0.09,test_accuracy:97.25%
